In [9]:
!pip install -q -U bitsandbytes peft transformers accelerate datasets

In [11]:
!pip install -q evaluate rouge_score sacrebleu


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 8.0 MB/s eta 0:00:00


In [2]:
from google.colab import files
import os

# Upload the file
print("Please upload 'synthetic_qa_pairs.jsonl'...")
uploaded = files.upload()

# Verify upload
for filename in uploaded.keys():
    print(f"User uploaded file: {filename}")

Please upload 'synthetic_qa_pairs.jsonl'...


Saving urdu_covid_passages_min.jsonl to urdu_covid_passages_min (1).jsonl
Saving urdu_covid_passages.tsv to urdu_covid_passages (1).tsv
Saving urdu_covid_corpus_clean.jsonl to urdu_covid_corpus_clean (1).jsonl
Saving synthetic_qa_pairs.jsonl to synthetic_qa_pairs (1).jsonl
Saving eval_queries.jsonl to eval_queries (1).jsonl
Saving hard_negatives.jsonl to hard_negatives (1).jsonl
User uploaded file: urdu_covid_passages_min (1).jsonl
User uploaded file: urdu_covid_passages (1).tsv
User uploaded file: urdu_covid_corpus_clean (1).jsonl
User uploaded file: synthetic_qa_pairs (1).jsonl
User uploaded file: eval_queries (1).jsonl
User uploaded file: hard_negatives (1).jsonl


In [3]:
import torch
import json
import os
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,                 # Using the standard Trainer
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Setup
model_id = "Qwen/Qwen2.5-1.5B-Instruct" # Lightweight model
new_model_name = "Qwen-Urdu-Generator"

# Fix connection timeouts
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "600"

In [4]:
import json

# 1. Load Tokenizer & Model (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(f"✅ Loaded {model_id} successfully.")

# 2. Load Data
data = []
file_path = "synthetic_qa_pairs.jsonl"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    print(f"✅ Loaded {len(data)} training examples.")
except FileNotFoundError:
    print("❌ File not found. Please run Cell 3 to upload.")
    data = []

# Create Dataset
dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

✅ Loaded Qwen/Qwen2.5-1.5B-Instruct successfully.
✅ Loaded 500 training examples.


In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. Prepare Model for Training (Crucial Fix)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# 2. Attach LoRA Adapters (This fixes the ValueError)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters() # Verifies that adapters are attached

# 3. Tokenize Data
def format_and_tokenize(example):
    ans = example.get('gold_answer', example.get('answer', ''))
    messages = [
        {"role": "system", "content": "You are an expert medical assistant. Answer in Urdu."},
        {"role": "user", "content": example['query']},
        {"role": "assistant", "content": ans}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False)

    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Processing data...")
tokenized_train = dataset["train"].map(format_and_tokenize)
tokenized_test = dataset["test"].map(format_and_tokenize)

# 4. Run Training
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

print("🚀 Starting Training...")
trainer.train()
print("✅ Training Complete.")

trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820
Processing data...


Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

🚀 Starting Training...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.864448
20,1.168480
30,1.003095
40,0.873675
50,0.762637


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


✅ Training Complete.


In [ ]:
from google.colab import files
import os

# Check if file exists, if not, prompt upload
if not os.path.exists("urdu_covid_corpus_clean.jsonl"):
    print("⚠️ Please upload 'urdu_covid_corpus_clean.jsonl' to get the reference answers.")
    files.upload()

In [12]:
import json
import torch
import evaluate
from tqdm import tqdm
from datasets import Dataset

# 1. Load the Ground Truth Answers (Corpus)
corpus = {}
print("Loading Corpus...")
with open("urdu_covid_corpus_clean.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        doc = json.loads(line)
        # Store ID -> Text mapping
        doc_id = doc.get('_id', doc.get('id'))
        doc_text = doc.get('text', doc.get('contents', doc.get('content', '')))
        if doc_id:
            corpus[doc_id] = doc_text

# 2. Prepare the Test Data
test_data = []
# We use the synthetic pairs you uploaded earlier
with open("synthetic_qa_pairs.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        pair = json.loads(line)
        pid = pair.get('positive_id')

        # Only keep pairs where we can find the real answer
        if pid in corpus:
            test_data.append({
                "query": pair['query'],
                "reference": corpus[pid] # The Ground Truth
            })

# Select just 5-10 samples for a quick test
test_subset = test_data[:10]
print(f"✅ Ready to evaluate on {len(test_subset)} samples.")

# 3. Setup Metrics
rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

predictions = []
references = []

model.eval()

print("⚡ Generating Answers...")
for item in tqdm(test_subset):
    query = item['query']
    ref = item['reference']

    # Format Prompt
    messages = [
        {"role": "system", "content": "You are an expert medical assistant. Answer in Urdu."},
        {"role": "user", "content": query}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=60,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("assistant")[-1].strip()

    predictions.append(response)
    references.append(ref)

    # Print 1 example to verify it's working
    if len(predictions) == 1:
        print(f"\n🔹 Query: {query}")
        print(f"🤖 Model: {response}")
        print(f"✅ Truth: {ref[:100]}...\n")

# 4. Calculate Scores
print("Computing Metrics...")
rouge_res = rouge.compute(predictions=predictions, references=references)
bleu_res = bleu.compute(predictions=predictions, references=references)

print("\n" + "="*30)
print("📊 FINAL RESULTS")
print("="*30)
print(f"ROUGE-L: {rouge_res['rougeL']:.4f}")
print(f"BLEU:    {bleu_res['bleu']:.4f}")
print("="*30)

Loading Corpus...
✅ Ready to evaluate on 10 samples.


⚡ Generating Answers...


 10%|█         | 1/10 [00:12<01:48, 12.03s/it]


🔹 Query: کووڈ-19 کی علامات میں کون سی چیزیں عام ہیں؟
🤖 Model: کووڈ-19 کے علامات کا استعمال کریں:

1. صحت کی ضرورت: اسکرینینگ، صحت کے لیے کن منظوری رپورٹینگ و
✅ Truth: کورونا وائرس مرض 2019 (COVID-19) ایک متعدی بیماری ہے جو SARS-CoV-2 وائرس کی وجہ سے ہوتی ہے۔ اس مرض ک...



100%|██████████| 10/10 [01:10<00:00,  7.09s/it]

Computing Metrics...

📊 FINAL RESULTS
ROUGE-L: 0.2105
BLEU:    0.0215
